<a href="https://colab.research.google.com/github/Jaimemorillo/ShouldIwatchThisMovie/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!cp /content/gdrive/MyDrive/methods.py .

In [ ]:
import unittest
import pandas as pd
import numpy as np
import methods as f

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

In [ ]:
original_df = 'gdrive/My Drive/TFG/overviews_final.csv'
stop_words_df = 'gdrive/My Drive/TFG/stopwords-es.txt'
testing_df = 'gdrive/My Drive/TFG/overviews_testing.csv'

In [ ]:
class TestDataset(unittest.TestCase):
  
  def setUp(self):
    '''Set up an instance pior to every test execution'''
    self.df = f.read_data(original_df)

  def test_label_and_overview_column(self):
    '''Test case function for label column'''
    self.assertTrue('like' in self.df.columns)
    self.assertTrue('overview' in self.df.columns)
      
  def test_enough_labels(self):
    '''Test case function for enough labels'''
    like_1  = self.df.like.value_counts()[1]
    like_0 = self.df.like.value_counts()[0]
    self.assertIn(like_1, range(800,1200))
    self.assertIn(like_0, range(800,1200))
  
  def test_empty_overview(self):
    '''Test case function for empty overview'''
    self.assertFalse(any(x == True for x in self.df['n_words']<5))

unittest.main(argv=[''], defaultTest='TestDataset', verbosity=2, exit=False)

test_empty_overview (__main__.TestDataset)
Test case function for empty overview ... ok
test_enough_labels (__main__.TestDataset)
Test case function for enough labels ... ok
test_label_and_overview_column (__main__.TestDataset)
Test case function for label column ... ok

----------------------------------------------------------------------
Ran 3 tests in 1.748s

OK


In [ ]:
class TestPreprocessing(unittest.TestCase):
  
  def setUp(self):
    '''Set up an instance pior to every test execution'''
    self.sentence = 'La princesa Leia, líder del movimiento rebelde que desea reinstaurar la República.\@¿?$%'
    self.new_sentece = f.preprocessing(self.sentence)

  def test_normalize(self):
    '''Test case function for normalize'''
    def is_ascii(s):
      return all(ord(c) < 128 for c in s)
    self.assertTrue(self.new_sentece.islower())
    self.assertTrue(is_ascii(self.new_sentece))
      
  def test_stop_words(self):
    '''Test case function for stop words'''
    stop_words = pd.read_csv(stop_words_df,header=None)
    stop_words = stop_words[0].tolist()
    stop_words = [word.lower() for word in stop_words]
    words = self.new_sentece.split(' ')
    self.assertFalse(any([word for word in words if word in stop_words]))

unittest.main(argv=[''], defaultTest='TestPreprocessing', verbosity=2, exit=False)

test_normalize (__main__.TestPreprocessing)
Test case function for normalize ... ok
test_stop_words (__main__.TestPreprocessing)
Test case function for stop words ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.034s

OK


In [ ]:
class TestSplit(unittest.TestCase):
  
  def setUp(self):
    '''Set up an instance pior to every test execution'''
    self.df = f.read_data(testing_df)
    self.X_train, self.X_test, self.y_train, self.y_test = f.split_train_test(self.df)

  def test_prop(self):
    '''Test case function for proportion'''
    train_len = len(self.X_train)
    test_len = len(self.X_test)
    len_df = len(self.df)
    prop_train = round(train_len/len_df, 2)
    prop_test = round(test_len/len_df, 2)
    self.assertEqual(prop_train, 0.7)
    self.assertEqual(prop_test, 0.3)

  def test_prop_label(self):
    '''Test case function for label proportion'''
    prop_train = round(self.y_train.value_counts(normalize=True)[1],1)
    prop_test = round(self.y_test.value_counts(normalize=True)[1], 1)
    self.assertEqual(prop_train, prop_test)

unittest.main(argv=[''], defaultTest='TestSplit', verbosity=2, exit=False)

test_prop (__main__.TestSplit)
Test case function for proportion ... ok
test_prop_label (__main__.TestSplit)
Test case function for label proportion ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.039s

OK


In [ ]:
class TestTokenize(unittest.TestCase):

  def setUp(self):
    '''Set up an instance pior to every test execution'''
    self.df = f.read_data(testing_df)
    self.X_train, self.X_test, self.y_train, self.y_test = f.split_train_test(self.df)
    self.X_train, self.X_test, self.vocab_size, self.real_maxlen = f.tokenize(self.X_train, self.X_test, test_mode=True)

  def test_array_of_ints(self):
    '''Test case function for array of ints'''
    self.assertEqual(self.X_train.dtype, 'int32')
  def test_max_len(self):
    '''Test case function for max len'''
    self.assertEqual(len(self.X_train[0]), self.real_maxlen)
  def test_vocab_size(self):
    '''Test case function for vocab size'''
    self.assertEqual(np.amax(self.X_train), self.vocab_size - 1)

unittest.main(argv=[''], defaultTest='TestTokenize', verbosity=2, exit=False)

test_array_of_ints (__main__.TestTokenize)
Test case function for array of ints ... ok
test_max_len (__main__.TestTokenize)
Test case function for max len ... ok
test_vocab_size (__main__.TestTokenize)
Test case function for vocab size ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.114s

OK


In [ ]:
class TestModel(unittest.TestCase):

  def setUp(self):
    '''Set up an instance pior to every test execution'''
    self.df = f.read_data(testing_df)
    self.X_train, self.X_test, self.y_train, self.y_test = f.split_train_test(self.df)
    self.X_train, self.X_test, self.vocab_size, self.real_maxlen = f.tokenize(self.X_train, self.X_test, test_mode=True)
    self.model = f.my_model(maxlen=self.real_maxlen, embedding_dim=5, 
                            vocab_size=self.vocab_size, lr=0.001, epochs=1)

  def test_model_structure(self):
    '''Test case function for model structure'''
    layers_names = [self.model.get_layer(index=i).name for i in range(0,len(self.model.layers))]
    self.assertTrue(any("conv" in s for s in layers_names))
    self.assertTrue(any("lstm" in s for s in layers_names) 
    or any("bidirectional" in s for s in layers_names))
    self.assertTrue(any("embedding" in s for s in layers_names))
    self.assertTrue(any("dense" in s for s in layers_names))
  
  def test_training(self):
    '''Test case function for model training'''
    history = self.model.fit(self.X_train, self.y_train, epochs=1, batch_size=len(self.X_train))
    self.assertTrue(len(history.history)>0)

  def test_predictions(self):
    '''Test case function for model predictions'''
    self.model.fit(self.X_train, self.y_train, epochs=1, batch_size=len(self.X_train))
    y_pred = self.model.predict(self.X_test).flatten()
    max_pred = y_pred.max()
    min_pred = y_pred.min()
    self.assertTrue((0<=max_pred<=1) and (0<=min_pred<=1))
    self.assertEqual(len(y_pred), len(self.y_test))

unittest.main(argv=[''], defaultTest='TestModel', verbosity=2, exit=False)

test_model_structure (__main__.TestModel)
Test case function for model structure ... ok
test_predictions (__main__.TestModel)
Test case function for model predictions ... 

1/1 [==============================] - 4s 4s/step - loss: 0.8375 - accuracy: 0.6000


ok
test_training (__main__.TestModel)
Test case function for model training ... 

1/1 [==============================] - 4s 4s/step - loss: 1.0384 - accuracy: 0.4714


ok

----------------------------------------------------------------------
Ran 3 tests in 11.935s

OK
